In [2]:
import pandas as pd
import numpy as np
import json
import datetime
import os
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
df_original_reports = pd.read_csv("data-processed/rki-reports.csv")
df_apple = pd.read_csv("data-processed/data_apple_prepared.csv")
df_rki_orig = pd.read_csv('../covid-19-dashboard/data/data_rki_apple_prepared_dash.csv')
df_jh_world = pd.read_csv('../covid-19-dashboard/data/data_jhu_world.csv')
# df = pd.read_csv("dash/data/data_rki_apple_prepared_dash.csv")
json_geo_de = json.load(open('../covid-19-dashboard/data/data_geo_de.json', 'r'))
df_rki_orig['date']=df_rki_orig['date'].astype('datetime64[ns]')
df_rki_orig.index = df_rki_orig.date
df_jh_world['date']=df_jh_world['date'].astype('datetime64[ns]')
df_jh_world.index = df_jh_world.date

In [4]:
df_jh_world

,land,iso_code,date,confirmed,confirmed_change,confirmed_active_cases,confirmed_active_cases_change,confirmed_peak_date,population_100k,confirmed_per_100k,confirmed_change_per_100k,confirmed_active_cases_per_100k,dead,dead_change,dead_doubling_days,lethality,dead_per_100k,dead_change_per_100k,region_wb
date,,,,,,,,,,,,,,,,,,,
2020-02-24,Afghanistan,AFG,2020-02-24,1.0,NaN,NaN,NaN,-1,371.72,0.002690,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,South Asia
2020-02-25,Afghanistan,AFG,2020-02-25,1.0,0.0,NaN,NaN,0,371.72,0.002690,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,South Asia
2020-02-26,Afghanistan,AFG,2020-02-26,1.0,0.0,NaN,NaN,0,371.72,0.002690,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,South Asia
2020-02-27,Afghanistan,AFG,2020-02-27,1.0,0.0,NaN,NaN,0,371.72,0.002690,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,South Asia
2020-02-28,Afghanistan,AFG,2020-02-28,1.0,0.0,NaN,NaN,0,371.72,0.002690,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,South Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-02,Zimbabwe,ZWE,2020-06-02,206.0,3.0,155.0,0.0,0,144.39,1.426692,0.02,1.073482,4.0,0.0,100.0,0.0,0.027703,0.0,Sub-Saharan Africa
2020-06-03,Zimbabwe,ZWE,2020-06-03,222.0,16.0,171.0,16.0,0,144.39,1.537503,0.11,1.184293,4.0,0.0,100.0,0.0,0.027703,0.0,Sub-Saharan Africa
2020-06-04,Zimbabwe,ZWE,2020-06-04,237.0,15.0,181.0,10.0,0,144.39,1.641388,0.10,1.253549,4.0,0.0,100.0,0.0,0.027703,0.0,Sub-Saharan Africa


In [49]:
# df_original_reports #.loc[df_original_reports.date != '2020-05-28'].drop("Unnamed: 0", axis=1).to_csv("data-processed/rki-reports.csv", index=False)

In [5]:
import pyarrow as pa
import redis

redis_conf = {'host': 'localhost', "port": 6379, 'db': 0}

redis_pool = None

def init():
    global redis_pool
    print("PID %d: initializing redis pool..." % os.getpid())
    redis_pool = redis.ConnectionPool(host=redis_conf['host'], port=redis_conf['port'], db=redis_conf['db'])

def cache_df(alias,df):

    cur = redis.Redis(connection_pool=redis_pool)
    context = pa.default_serialization_context()
    df_compressed =  context.serialize(df).to_buffer().to_pybytes()

    res = cur.set(alias,df_compressed)
    if res == True:
        print('df cached')

In [6]:
init()

PID 79100: initializing redis pool...


In [7]:
cache_df('df_jh_world', df_jh_world)
cache_df('df_rki_orig', df_rki_orig)
cache_df('json_geo_de', json_geo_de)

df cached
df cached
df cached


In [53]:
redis_pool.disconnect()

In [21]:
def get_cached_df(alias):

    cur = redis.Redis(connection_pool=redis_pool)
    context = pa.default_serialization_context()
    all_keys = [key.decode("utf-8") for key in cur.keys()]

    if alias in all_keys:   
        result = cur.get(alias)

        dataframe = pd.DataFrame.from_dict(context.deserialize(result))

        return dataframe.copy()

    return None

In [27]:
def get_cached_json(alias):

    cur = redis.Redis(connection_pool=redis_pool)
    context = pa.default_serialization_context()
    all_keys = [key.decode("utf-8") for key in cur.keys()]

    if alias in all_keys:
        result = cur.get(alias)

        json_result = context.deserialize(result)

        return json_result

    return None

In [22]:
df = get_cached_df('df_jh_world')

In [23]:
df['date'] = df['date'].astype('datetime64[ns]')

In [29]:
type( get_cached_json('json_geo_de'))

dict